# 1.st model

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import warnings

warnings.filterwarnings('ignore')
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,StoppingCriteria, StoppingCriteriaList
import torch
import threading

In [15]:
class Stream(StoppingCriteria):
    def __init__(self, tokenizer, stop_token_id=None):
        self.tokenizer = tokenizer
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        new_token_id = input_ids[0, -1].item()
        if self.stop_token_id is not None and new_token_id == self.stop_token_id:
            return True
        token_text = self.tokenizer.decode([new_token_id], skip_special_tokens=False)
        print(token_text, end="", flush=True)
        return False
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def loading_pdf(path="/home/pa2/ml/Document-Classification/files/1_D:\BPL\BLS\Document\Arbitration Award of Zostel Hospitality relied upon by the Claimant.txt"):
    """Loads a text file and splits it into chunks."""
    loader = TextLoader(path, encoding="utf-8")
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=100
    )
    docs = text_splitter.split_documents(docs)
    
    return docs
    
def faiss1(docs):
    model=HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2"
    )
    db=FAISS.from_documents(docs,model)
    return db
def model1(context,query,chat,model,tokenizer,maxlen=3000):
    
    message = [{
        "role": "user",
        "content": f"""You are a legal assistant trained to classify legal documents with zero tolerance for guessing. You will follow the rules below without exception:

RULES:
1. Do NOT make assumptions or use external knowledge.
2. You MUST only use the words or phrases found directly in the context.
3. You MUST explain your classification using legal terminology that appears in the context.
4. You MUST follow the exact output format shown below.
5. You MUST return one and only one legal category from the provided list.

OUTPUT FORMAT:

<think>
Step-by-step reasoning using ONLY the context. Refer to legal phrases or structure. Do not go outside the context or speculate.
</think>

<answer>
Choose ONLY ONE from this list (copy-paste one line only):

1. Contractual  
2. Regulatory and Compliance  
3. Intellectual Property Rights (IPR)  
4. Litigation and Dispute  
5. Property and Real Estate  
6. Corporate and Governance  
7. Employment and Human Resources (HR)  
8. Finance and Banking  
9. Government and Public Laws  
10. Personal and Civil Legal Documents  
11. Others

The answer must exactly match one of the above. Do NOT add anything else.
</answer>

<context>
{context}
</context>

Question: What is the correct legal category for this document?

"""


    }]

    prompt=tokenizer.apply_chat_template(message,tokenize=False)
    input_ids=tokenizer(prompt,return_tensors='pt').to("cuda")
    stopping_criteria = StoppingCriteriaList([Stream(tokenizer)])
    with torch.no_grad():
        response=model.generate(input_ids=input_ids['input_ids'],
                                attention_mask=input_ids['attention_mask'],
                                stopping_criteria=stopping_criteria,
                                max_new_tokens=maxlen,
                                use_cache=True,
                               temperature=0.1,)
    answer=tokenizer.decode(response[0])
    print()
    return answer
def loading_pdf1(path):
    """Loads a PDF or text file and splits it into chunks."""
    if path.endswith(".txt"):
        loader = TextLoader(path, encoding="utf-8")
    elif path.endswith(".pdf"):
        loader = PyPDFLoader(path)
    else:
        raise ValueError("Unsupported file type. Only .txt and .pdf are supported.")

    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    docs = text_splitter.split_documents(docs)
    return docs
# docs=loading_pdf()
docs=loading_pdf1("/home/pa2/ml/Document-Classification/files/US7054127.pdf")
db=faiss1(docs)

OutOfMemoryError: CUDA out of memory. Tried to allocate 30.00 MiB. GPU 0 has a total capacity of 7.78 GiB of which 18.62 MiB is free. Process 1482 has 124.41 MiB memory in use. Including non-PyTorch memory, this process has 7.63 GiB memory in use. Of the allocated memory 7.45 GiB is allocated by PyTorch, and 117.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [3]:
tokenizer = AutoTokenizer.from_pretrained('deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

model = AutoModelForCausalLM.from_pretrained(
    'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
    device_map='cuda',
    quantization_config=bnb_config,
)

In [11]:
if __name__ == '__main__':
    chat = ""
    
    while True:
        query = "Identify the type of this legal document based on its content. Return only the most informative parts for classification like headers, definitions, or legal phrases."

        if query.lower() == 'exit':
            print("Thanks for using the classifier!")
            break
        similar_chunks = db.similarity_search(query, k=5)
        context = "\n\n".join([doc.page_content for doc in similar_chunks])
        try:
            response = model1(context=context, query=query, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
            print("\n[Model Output]:\n", response.strip(), "\n")
            chat = response

        except Exception as e:
            print(f"[Error during inference]: {e}")

        torch.cuda.empty_cache()
        break

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Options:
1. Contractual
2. Regulatory and Compliance
3. Intellectual Property Rights (IPR)
4. Litigation and Dispute
5. Property and Real Estate
6. Corporate and Governance
7. Employment and Human Resources (HR)
8. Finance and Banking
9. Government and Public Laws
10. Personal and Civil Legal Documents
11. Others

Answer: Personal and Civil Legal Documents<｜end▁of▁sentence｜>

[Model Output]:
 <｜begin▁of▁sentence｜><｜begin▁of▁sentence｜><｜User｜>You are a legal assistant trained to classify legal documents with zero tolerance for guessing. You will follow the rules below without exception:

RULES:
1. Do NOT make assumptions or use external knowledge.
2. You MUST only use the words or phrases found directly in the context.
3. You MUST explain your classification using legal terminology that appears in the context.
4. You MUST follow the exact output format shown below.
5. You MUST return one and only one legal category from the provided list.

OUTPUT FORMAT:

<think>
Step-by-step reasoning 

# Using LLAMA 3 8B

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import warnings

warnings.filterwarnings('ignore')
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,StoppingCriteria, StoppingCriteriaList
import torch
import threading

In [2]:
class Stream(StoppingCriteria):
    def __init__(self, tokenizer, stop_token_id=None):
        self.tokenizer = tokenizer
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        new_token_id = input_ids[0, -1].item()
        if self.stop_token_id is not None and new_token_id == self.stop_token_id:
            return True
        token_text = self.tokenizer.decode([new_token_id], skip_special_tokens=False)
        print(token_text, end="", flush=True)
        return False
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def loading_pdf(path="/home/pa2/ml/Document-Classification/files/1_D:\BPL\BLS\Document\Arbitration Award of Zostel Hospitality relied upon by the Claimant.txt"):
    """Loads a text file and splits it into chunks."""
    loader = TextLoader(path, encoding="utf-8")
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=100
    )
    docs = text_splitter.split_documents(docs)
    
    return docs
    
def faiss1(docs):
    model=HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2"
    )
    db=FAISS.from_documents(docs,model)
    return db
def model1(context,query,chat,model,tokenizer,maxlen=3000):
    
    message = [{
        "role": "user",
        "content": f"""You are a legal assistant trained to classify legal documents with zero tolerance for guessing. You will follow the rules below without exception:

RULES:
1. Do NOT make assumptions or use external knowledge.
2. You MUST only use the words or phrases found directly in the context.
3. You MUST explain your classification using legal terminology that appears in the context.
4. You MUST follow the exact output format shown below.
5. You MUST return one and only one legal category from the provided list.

OUTPUT FORMAT:

<think>
Step-by-step reasoning using ONLY the context. Refer to legal phrases or structure. Do not go outside the context or speculate.
</think>

<answer>
Choose ONLY ONE from this list (copy-paste one line only):

1. Contractual  
2. Regulatory and Compliance  
3. Intellectual Property Rights (IPR)  
4. Litigation and Dispute  
5. Property and Real Estate  
6. Corporate and Governance  
7. Employment and Human Resources (HR)  
8. Finance and Banking  
9. Government and Public Laws  
10. Personal and Civil Legal Documents  
11. Others

The answer must exactly match one of the above. Do NOT add anything else.
</answer>

<context>
{context}
</context>

Question: What is the correct legal category for this document?

"""


    }]

    prompt=tokenizer.apply_chat_template(message,tokenize=False)
    input_ids=tokenizer(prompt,return_tensors='pt').to("cuda")
    stopping_criteria = StoppingCriteriaList([Stream(tokenizer)])
    with torch.no_grad():
        response=model.generate(input_ids=input_ids['input_ids'],
                                attention_mask=input_ids['attention_mask'],
                                stopping_criteria=stopping_criteria,
                                max_new_tokens=maxlen,
                                use_cache=True,
                               temperature=0.1,)
    answer=tokenizer.decode(response[0])
    print()
    return answer
def loading_pdf1(path):
    """Loads a PDF or text file and splits it into chunks."""
    if path.endswith(".txt"):
        loader = TextLoader(path, encoding="utf-8")
    elif path.lower().endswith(".pdf"):
        loader = PyPDFLoader(path)
    else:
        raise ValueError("Unsupported file type. Only .txt and .pdf are supported.")

    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    docs = text_splitter.split_documents(docs)
    return docs
# docs=loading_pdf()
docs=loading_pdf1("/home/pa2/ml/Document-Classification/files/US7054127.pdf")
db=faiss1(docs)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    device_map='cuda',
    quantization_config=bnb_config,
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
if __name__ == '__main__':
    chat = ""
    
    while True:
        query = "Identify the type of this legal document based on its content. Return only the most informative parts for classification like headers, definitions, or legal phrases."

        if query.lower() == 'exit':
            print("Thanks for using the classifier!")
            break
        similar_chunks = db.similarity_search(query, k=5)
        context = "\n\n".join([doc.page_content for doc in similar_chunks])
        try:
            response = model1(context=context, query=query, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
            chat = response

        except Exception as e:
            print(f"[Error during inference]: {e}")

        torch.cuda.empty_cache()
        break

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

<think>
The document is a patent application or patent, as indicated by the "U.S. Patent" label and the presence of a patent number (7,054,127 B1). The content describes a technical invention, including a detailed explanation of the invention, its purpose, and its classification under the International Classification (Int. Cl.) and U.S. Classification (U.S. Cl.). The document also mentions the Primary Examiner and Assistant Examiner, which are roles associated with the patent examination process. Furthermore, the document includes a notice of extension or adjustment of the patent term under 35 U.S.C. 154(b). These elements are characteristic of a patent document.

</think>

<answer>
1. Contractual  
</answer><|eot_id|>


# LLama 3.2 3B

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import warnings

warnings.filterwarnings('ignore')
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,StoppingCriteria, StoppingCriteriaList
import torch
import threading

In [2]:
class Stream(StoppingCriteria):
    def __init__(self, tokenizer, stop_token_id=None):
        self.tokenizer = tokenizer
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        new_token_id = input_ids[0, -1].item()
        if self.stop_token_id is not None and new_token_id == self.stop_token_id:
            return True
        token_text = self.tokenizer.decode([new_token_id], skip_special_tokens=False)
        print(token_text, end="", flush=True)
        return False
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def loading_pdf(path="/home/pa2/ml/Document-Classification/files/1_D:\BPL\BLS\Document\Arbitration Award of Zostel Hospitality relied upon by the Claimant.txt"):
    """Loads a text file and splits it into chunks."""
    loader = TextLoader(path, encoding="utf-8")
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=100
    )
    docs = text_splitter.split_documents(docs)
    
    return docs
    
def faiss1(docs):
    model=HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2"
    )
    db=FAISS.from_documents(docs,model)
    return db
def model1(context,query,chat,model,tokenizer,maxlen=3000):
    
    message = [{
        "role": "user",
        "content": f"""You are a legal assistant trained to classify legal documents with zero tolerance for guessing. You must follow the rules below strictly:

RULES:
1. Do NOT make assumptions or use external knowledge.
2. ONLY use exact words, legal terms, and structures from the given context.
3. You MUST explain your classification using legal terminology from the context.
4. You MUST follow the exact output format below.
5. Return one and only one category from the list below.

LEGAL CATEGORIES with EXAMPLES:

1. Contractual – (e.g., B2B agreements, purchase orders, NDAs, service contracts)  
2. Regulatory and Compliance – (e.g., compliance reports, internal audits, regulatory filings)  
3. Intellectual Property Rights (IPR) – (e.g., patents, trademarks, copyrights, design rights, trade secrets)  
4. Litigation and Dispute – (e.g., court filings, legal notices, arbitration claims, judgments)  
5. Property and Real Estate – (e.g., lease agreements, sale deeds, land registration, rent contracts)  
6. Corporate and Governance – (e.g., company formation docs, resolutions, board meeting records, annual filings)  
7. Employment and Human Resources (HR) – (e.g., employee contracts, HR policies, separation letters)  
8. Finance and Banking – (e.g., loan agreements, collateral deeds, investment deals, bank guarantees)  
9. Government and Public Laws – (e.g., RTI responses, government licenses, public notices, certificates)  
10. Personal and Civil Legal Documents – (e.g., ID proofs, affidavits, marriage/divorce documents, family law papers)  
11. Others – (Only if no other category fits)

---
MANDATORY RULE MUST FOLLOW :-
-> **PROPERLY SHOW ALL THE STEPS IN <think> SECTION** (MANDATORY)
<context>
{context}
</context>

---

<think>Steps

Step 1: Extracted Legal Keywords →  
[List all legal terms, phrases, and document structures found directly in the context.]

Step 2: Short Legal Summary →  
[Summarize the context in 2–3 lines using legal wording, sticking strictly to the content.]

Step 3: Multi-Angle Rethinking →  
[Restate the summary from multiple angles: formal, casual, detailed, abstract, technical.]

Step 4: Category Matching →  
[Compare summary + keywords against each category. Write your thought process for every match or mismatch.]

Step 5: Eliminate Unrelated Categories →  
[List all categories that clearly do NOT apply and briefly explain why.]

Step 6: Deep Legal Reasoning →  
[Based on the remaining options, explain in detail why one specific category is the strongest fit using legal logic.]

Step 7: Final Justification →  
[Final 1-2 lines defending your classification using exact phrases or structure from the context.]

</think>

<answer>
Choose ONLY ONE from this list (copy-paste one line only):

1. Contractual  
2. Regulatory and Compliance  
3. Intellectual Property Rights (IPR)  
4. Litigation and Dispute  
5. Property and Real Estate  
6. Corporate and Governance  
7. Employment and Human Resources (HR)  
8. Finance and Banking  
9. Government and Public Laws  
10. Personal and Civil Legal Documents  
11. Others
</answer>


Question: What is the correct legal category for this document?
"""


    }]

    prompt=tokenizer.apply_chat_template(message,tokenize=False)
    input_ids=tokenizer(prompt,return_tensors='pt').to("cuda")
    stopping_criteria = StoppingCriteriaList([Stream(tokenizer)])
    with torch.no_grad():
        response=model.generate(input_ids=input_ids['input_ids'],
                                attention_mask=input_ids['attention_mask'],
                                stopping_criteria=stopping_criteria,
                                max_new_tokens=maxlen,
                                use_cache=True,
                               temperature=0.1,)
    answer=tokenizer.decode(response[0])
    print()
    return answer
def loading_pdf1(path):
    """Loads a PDF or text file and splits it into chunks."""
    if path.endswith(".txt"):
        loader = TextLoader(path, encoding="utf-8")
    elif path.lower().endswith(".pdf"):
        loader = PyPDFLoader(path)
    else:
        raise ValueError("Unsupported file type. Only .txt and .pdf are supported.")

    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=20
    )
    docs = text_splitter.split_documents(docs)
    return docs
# docs=loading_pdf()
docs=loading_pdf1("/home/pa2/ml/Document-Classification/Dataset/Litigation and Dispute/Master_G_Through_Legal_Guardian_Anr_vs_State_Nct_Of_Delhi_Home_Department_on_2_April_2025.PDF")
db=faiss1(docs)

In [3]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-3B-Instruct')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-3.2-3B-Instruct',
    device_map='cuda',
    quantization_config=bnb_config,
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
if __name__ == '__main__':
    chat = ""
    
    while True:
        query = """Retrieve the most informative sections of this legal document that can help determine its legal classification. Prioritize segments containing legal terms, section headings, clauses, named parties, obligations, dates, and governing laws. Specifically look for any signs related to:

1. Contractual – references to B2B or B2C agreements, terms and conditions, service-level agreements, non-disclosure agreements, etc.
2. Regulatory and Compliance – internal audit language, compliance obligations, policy enforcement, external submissions to government bodies.
3. Intellectual Property Rights (IPR) – mentions of patents, copyrights, trademarks, trade secrets, licenses, or intellectual asset ownership.
4. Litigation and Dispute – phrases like "court order", "petition", "judgment", "arbitration", "claim", or "legal dispute".
5. Property and Real Estate – lease terms, ownership transfers, land records, mortgage agreements, property deeds.
6. Corporate and Governance – company founding documents, board resolutions, shareholder agreements, filings, annual reports.
7. Employment and HR – employment agreements, HR policies, termination letters, NDAs, resignation or separation terms.
8. Finance and Banking – loan terms, guarantee clauses, investment contracts, collateral agreements, promissory notes.
9. Government and Public Laws – public notices, licenses, Right to Information (RTI) references, certificates issued by authorities.
10. Personal and Civil Legal Documents – ID proofs, marriage/divorce certificates, affidavits, inheritance documents, family law terms.
11. Others – anything that doesn’t fit above but still contains legal relevance.

Return the most relevant parts verbatim that clearly show which category the document may belong to.
"""

        if query.lower() == 'exit':
            print("Thanks for using the classifier!")
            break
        similar_chunks = db.similarity_search(query, k=5)
        context = "\n\n".join([doc.page_content for doc in similar_chunks])
        try:
            response = model1(context=context, query=query, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
            # print("\n[Model Output]:\n", response.strip(), "\n")
            chat = response

        except Exception as e:
            print(f"[Error during inference]: {e}")
        torch.cuda.empty_cache()
        break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

<answer>
1. Contractual<|eot_id|>


In [ ]:
# 1. Contractual  like B2B or B2C etc
# 2. Regulatory and Compliance  like Internal compliance and External regulatory submission etc
# 3. Intellectual Property Rights (IPR) like patent, copyright, trademarks , trade secrets, design etc
# 4. Litigation and Dispute  like pleadings and petitions, court orders and judegements , arbitration etc
# 5. Property and Real Estate  like ownership transfer,lease/rental,land records etc
# 6. Corporate and Governance  like Foundational document,corporate actions, annual repors and filings etc
# 7. Employment and Human Resources (HR)  like employee contracts, HR policy, seperation Docs(resignation) etc
# 8. Finance and Banking  loan documents, gurantees and collaterals, investment agreements etc
# 9. Government and Public Laws  licences, public notices, rti, certificates etc
# 10. Personal and Civil Legal Documents  Identiy, affidavites, family laws etc
# 11. Others

In [ ]:
# f"""You are a legal assistant that classifies legal documents based **strictly** on the provided context.


# Please follow this format exactly:


# <think>

# Step-by-step legal reasoning based **only** on the context.

#     1.Identify the **document type** (e.g., act, contract, judgment, petition) using only terms directly visible in the context.

#     2.List exact **legal terms, sections, or phrases** (e.g., "petition", "statute", "employment", "IPR", "compliance", "court", etc.) found in the text. Do not interpret — quote them exactly.

#     3.Check for **structural indicators**:

#        - Are there section numbers? → Could indicate a statute.

#        - Are there parties, rulings, judgments, or citations? → Could indicate litigation.

#        - Are there terms like "agreement", "obligations", "consideration"? → Could indicate contracts.

#     4.Eliminate each incorrect category **one by one**, using direct contradictions or lack of evidence.

#     5.Final Decision: Choose only the category that is **explicitly and unambiguously supported** by the language or structure in the context.

#     6.If you have to guess or cannot find conclusive evidence for any listed category, select **"11. Others"**.


# Remember:

# - You must behave like a legal auditor under oath.

# - Do not infer meaning, do not rely on general knowledge, and do not use legal intuition — only analyze what is present in the context.

# - Your reasoning must be slow, careful, and legally justifiable at every step.

# </think>

# <answer>

# Respond with only one category from this exact list — nothing else:


# 1. Contractual  

# 2. Regulatory and Compliance  

# 3. Intellectual Property Rights (IPR)  

# 4. Litigation and Dispute  

# 5. Property and Real Estate  

# 6. Corporate and Governance  

# 7. Employment and Human Resources (HR)  

# 8. Finance and Banking  

# 9. Government and Public Laws  

# 10. Personal and Civil Legal Documents

# 11. Others

# Your response must **exactly match one of these options** (e.g., `4. Litigation and Dispute`). Do **not** generate any answer outside this list.

# </answer>


# <context>

# {context}

# </context>


# Question: What is the correct legal category for this document?

# """ 

# LLama 3.2 1B (Good working)

# TEST

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import warnings

warnings.filterwarnings('ignore')
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,StoppingCriteria, StoppingCriteriaList
import torch
import threading

In [2]:
class Stream(StoppingCriteria):
    def __init__(self, tokenizer, stop_token_id=None):
        self.tokenizer = tokenizer
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        new_token_id = input_ids[0, -1].item()
        if self.stop_token_id is not None and new_token_id == self.stop_token_id:
            return True
        token_text = self.tokenizer.decode([new_token_id], skip_special_tokens=False)
        print(token_text, end="", flush=True)
        return False
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def loading_pdf(path="/home/pa2/ml/Document-Classification/files/1_D:\BPL\BLS\Document\Arbitration Award of Zostel Hospitality relied upon by the Claimant.txt"):
    """Loads a text file and splits it into chunks."""
    loader = TextLoader(path, encoding="utf-8")
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=100
    )
    docs = text_splitter.split_documents(docs)
    
    return docs
    
def faiss1(docs):
    model=HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2"
    )
    db=FAISS.from_documents(docs,model)
    return db
def model1(context,query,chat,model,tokenizer,maxlen=3000):
    
    message = [{
        "role": "user",
        "content": f"""You are a legal assistant trained to classify legal documents with zero tolerance for guessing. You must follow the rules below strictly:

RULES:
1. Do NOT make assumptions or use external knowledge.
2. ONLY use exact words, legal terms, and structures from the given context.
3. You MUST explain your classification using legal terminology from the context.
4. You MUST follow the exact output format below.
5. Return one and only one category from the list below.

LEGAL CATEGORIES with EXAMPLES:

1. Contractual – (e.g., B2B agreements, purchase orders, NDAs, service contracts)  
2. Regulatory and Compliance – (e.g., compliance reports, internal audits, regulatory filings)  
3. Intellectual Property Rights (IPR) – (e.g., patents, trademarks, copyrights, design rights, trade secrets)  
4. Litigation and Dispute – (e.g., court filings, legal notices, arbitration claims, judgments)  
5. Property and Real Estate – (e.g., lease agreements, sale deeds, land registration, rent contracts)  
6. Corporate and Governance – (e.g., company formation docs, resolutions, board meeting records, annual filings)  
7. Employment and Human Resources (HR) – (e.g., employee contracts, HR policies, separation letters)  
8. Finance and Banking – (e.g., loan agreements, collateral deeds, investment deals, bank guarantees)  
9. Government and Public Laws – (e.g., RTI responses, government licenses, public notices, certificates)  
10. Personal and Civil Legal Documents – (e.g., ID proofs, affidavits, marriage/divorce documents, family law papers)  
11. Others – (Only if no other category fits)

---
MANDATORY RULE MUST FOLLOW :-
-> **PROPERLY SHOW ALL THE STEPS IN <think> SECTION** (MANDATORY)
<context>
{context}
</context>

---

<think>Steps

Step 1: Extracted Legal Keywords →  
[List all legal terms, phrases, and document structures found directly in the context.]

Step 2: Short Legal Summary →  
[Summarize the context in 2–3 lines using legal wording, sticking strictly to the content.]

Step 3: Multi-Angle Rethinking →  
[Restate the summary from multiple angles: formal, casual, detailed, abstract, technical.]

Step 4: Category Matching →  
[Compare summary + keywords against each category. Write your thought process for every match or mismatch.]

Step 5: Eliminate Unrelated Categories →  
[List all categories that clearly do NOT apply and briefly explain why.]

Step 6: Deep Legal Reasoning →  
[Based on the remaining options, explain in detail why one specific category is the strongest fit using legal logic.]

Step 7: Final Justification →  
[Final 1-2 lines defending your classification using exact phrases or structure from the context.]

</think>

<answer>
Choose ONLY ONE from this list (copy-paste one line only):

1. Contractual  
2. Regulatory and Compliance  
3. Intellectual Property Rights (IPR)  
4. Litigation and Dispute  
5. Property and Real Estate  
6. Corporate and Governance  
7. Employment and Human Resources (HR)  
8. Finance and Banking  
9. Government and Public Laws  
10. Personal and Civil Legal Documents  
11. Others
</answer>


Question: What is the correct legal category for this document?
"""


    }]

    prompt=tokenizer.apply_chat_template(message,tokenize=False)
    input_ids=tokenizer(prompt,return_tensors='pt').to("cuda")
    stopping_criteria = StoppingCriteriaList([Stream(tokenizer)])
    with torch.no_grad():
        response=model.generate(input_ids=input_ids['input_ids'],
                                attention_mask=input_ids['attention_mask'],
                                stopping_criteria=stopping_criteria,
                                max_new_tokens=maxlen,
                                use_cache=True,
                               temperature=0.1,)
    answer=tokenizer.decode(response[0])
    print()
    return answer
def loading_pdf1(path):
    """Loads a PDF or text file and splits it into chunks."""
    if path.endswith(".txt"):
        loader = TextLoader(path, encoding="utf-8")
    elif path.lower().endswith(".pdf"):
        loader = PyPDFLoader(path)
    else:
        raise ValueError("Unsupported file type. Only .txt and .pdf are supported.")

    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=20
    )
    docs = text_splitter.split_documents(docs)
    return docs
# docs=loading_pdf()
docs=loading_pdf1("/home/pa2/ml/Document-Classification/files/AL6988A-BPL-20240601-001.pdf")
db=faiss1(docs)

In [3]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.1-8B-Instruct')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-3.1-8B-Instruct',
    device_map='cuda',
    quantization_config=bnb_config,
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
if __name__ == '__main__':
    chat = ""
    
    while True:
        query = """Retrieve the most informative sections of this legal document that can help determine its legal classification. Prioritize segments containing legal terms, section headings, clauses, named parties, obligations, dates, and governing laws. Specifically look for any signs related to:

1. Contractual – references to B2B or B2C agreements, terms and conditions, service-level agreements, non-disclosure agreements, etc.
2. Regulatory and Compliance – internal audit language, compliance obligations, policy enforcement, external submissions to government bodies.
3. Intellectual Property Rights (IPR) – mentions of patents, copyrights, trademarks, trade secrets, licenses, or intellectual asset ownership.
4. Litigation and Dispute – phrases like "court order", "petition", "judgment", "arbitration", "claim", or "legal dispute".
5. Property and Real Estate – lease terms, ownership transfers, land records, mortgage agreements, property deeds.
6. Corporate and Governance – company founding documents, board resolutions, shareholder agreements, filings, annual reports.
7. Employment and HR – employment agreements, HR policies, termination letters, NDAs, resignation or separation terms.
8. Finance and Banking – loan terms, guarantee clauses, investment contracts, collateral agreements, promissory notes.
9. Government and Public Laws – public notices, licenses, Right to Information (RTI) references, certificates issued by authorities.
10. Personal and Civil Legal Documents – ID proofs, marriage/divorce certificates, affidavits, inheritance documents, family law terms.
11. Others – anything that doesn’t fit above but still contains legal relevance.


Return the most relevant parts verbatim that clearly show which category the document may belong to.
"""

        if query.lower() == 'exit':
            print("Thanks for using the classifier!")
            break
        similar_chunks = db.similarity_search(query, k=5)
        context = "\n\n".join([doc.page_content for doc in similar_chunks])
        try:
            response = model1(context=context, query=query, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
            # print("\n[Model Output]:\n", response.strip(), "\n")
            chat = response

        except Exception as e:
            print(f"[Error during inference]: {e}")
        torch.cuda.empty_cache()
        break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

<think>

Step 1: Extracted Legal Keywords  
[List all legal terms, phrases, and document structures found directly in the context.]

- Invoice
- Letter of Credit (LC)
- Authorized Dealer (AD)
- High Sea Sale (HSS)
- Commission & Brokerage
- Cost of Container (CoC)
- Cost of Packing (CoP)
- Handling Charges (HND CHG)
- Goods and Service input cost (G&S)
- Document Charges (DOC CH)
- Country of Origin Certificate (CoO)
- Royalty and Licence Fees (R&LF)
- Loading Unloading Charges (LD/ULD)
- Warranty Services (WS)
- Other Costs (OTC)
- Customs Tariff Head (CTH)
- Unit Quantity Code (UQC)

Step 2: Short Legal Summary  
[Summarize the context in 2–3 lines using legal wording, sticking strictly to the content.]

This document appears to be a commercial invoice detailing goods and services provided, including charges and fees associated with their transportation and documentation.

Step 3: Multi-Angle Rethinking  
[Restate the summary from multip

In [5]:
print(response)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are a legal assistant trained to classify legal documents with zero tolerance for guessing. You must follow the rules below strictly:

RULES:
1. Do NOT make assumptions or use external knowledge.
2. ONLY use exact words, legal terms, and structures from the given context.
3. You MUST explain your classification using legal terminology from the context.
4. You MUST follow the exact output format below.
5. Return one and only one category from the list below.

LEGAL CATEGORIES with EXAMPLES:

1. Contractual – (e.g., B2B agreements, purchase orders, NDAs, service contracts)  
2. Regulatory and Compliance – (e.g., compliance reports, internal audits, regulatory filings)  
3. Intellectual Property Rights (IPR) – (e.g., patents, trademarks, copyrights, design rights, trade secrets)  
4. Litigation 

In [6]:
if __name__ == '__main__':
    chat = ""
    
    while True:
        query = """Retrieve the most informative sections of this legal document that can help determine its legal classification. Prioritize segments containing legal terms, section headings, clauses, named parties, obligations, dates, and governing laws. Specifically look for any signs related to:

1. Contractual – references to B2B or B2C agreements, terms and conditions, service-level agreements, non-disclosure agreements, etc.
2. Regulatory and Compliance – internal audit language, compliance obligations, policy enforcement, external submissions to government bodies.
3. Intellectual Property Rights (IPR) – mentions of patents, copyrights, trademarks, trade secrets, licenses, or intellectual asset ownership.
4. Litigation and Dispute – phrases like "court order", "petition", "judgment", "arbitration", "claim", or "legal dispute".
5. Property and Real Estate – lease terms, ownership transfers, land records, mortgage agreements, property deeds.
6. Corporate and Governance – company founding documents, board resolutions, shareholder agreements, filings, annual reports.
7. Employment and HR – employment agreements, HR policies, termination letters, NDAs, resignation or separation terms.
8. Finance and Banking – loan terms, guarantee clauses, investment contracts, collateral agreements, promissory notes.
9. Government and Public Laws – public notices, licenses, Right to Information (RTI) references, certificates issued by authorities.
10. Personal and Civil Legal Documents – ID proofs, marriage/divorce certificates, affidavits, inheritance documents, family law terms.
11. Others – anything that doesn’t fit above but still contains legal relevance.

Return the most relevant parts verbatim that clearly show which category the document may belong to.
"""

        if query.lower() == 'exit':
            print("Thanks for using the classifier!")
            break
        similar_chunks = db.similarity_search(query, k=5)
        context = "\n\n".join([doc.page_content for doc in similar_chunks])
        try:
            response = model1(context=context, query=query, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
            # print("\n[Model Output]:\n", response.strip(), "\n")
            chat = response

        except Exception as e:
            print(f"[Error during inference]: {e}")
        torch.cuda.empty_cache()
        break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

<think>

Step 1: Extracted Legal Keywords  
[Patent, U.S. Patent, U.S. Patent application, invention, embodiment, clamping circuit, components, flow chart, fabricating, internal conductors, wires, conductors, node, ground path, transient voltages, clamping circuit, components, transZorbs, fast-acting diodes, TVS Zener diodes, arrestor devices, metal oxide varistors, carbon blocks, thyristors, gas discharge tubes, cable, internal conductors, wires, conductors, node, ground path, transient voltages, clamping circuit, components, transZorbs, fast-acting diodes, TVS Zener diodes, arrestor devices, metal oxide varistors, carbon blocks, thyristors, gas discharge tubes]

Step 2: Short Legal Summary  
This document pertains to a U.S. Patent application describing an invention involving a clamping circuit and components for protecting against transient voltages.

Step 3: Multi-Angle Rethinking  
Formally, this document is a patent application. Casu

In [6]:
if __name__ == '__main__':
    chat = ""
    
    while True:
        query = """Retrieve the most informative sections of this legal document that can help determine its legal classification. Prioritize segments containing legal terms, section headings, clauses, named parties, obligations, dates, and governing laws. Specifically look for any signs related to:

1. Contractual – references to B2B or B2C agreements, terms and conditions, service-level agreements, non-disclosure agreements, etc.
2. Regulatory and Compliance – internal audit language, compliance obligations, policy enforcement, external submissions to government bodies.
3. Intellectual Property Rights (IPR) – mentions of patents, copyrights, trademarks, trade secrets, licenses, or intellectual asset ownership.
4. Litigation and Dispute – phrases like "court order", "petition", "judgment", "arbitration", "claim", or "legal dispute".
5. Property and Real Estate – lease terms, ownership transfers, land records, mortgage agreements, property deeds.
6. Corporate and Governance – company founding documents, board resolutions, shareholder agreements, filings, annual reports.
7. Employment and HR – employment agreements, HR policies, termination letters, NDAs, resignation or separation terms.
8. Finance and Banking – loan terms, guarantee clauses, investment contracts, collateral agreements, promissory notes.
9. Government and Public Laws – public notices, licenses, Right to Information (RTI) references, certificates issued by authorities.
10. Personal and Civil Legal Documents – ID proofs, marriage/divorce certificates, affidavits, inheritance documents, family law terms.
11. Others – anything that doesn’t fit above but still contains legal relevance.

Return the most relevant parts verbatim that clearly show which category the document may belong to.
"""

        if query.lower() == 'exit':
            print("Thanks for using the classifier!")
            break
        similar_chunks = db.similarity_search(query, k=5)
        context = "\n\n".join([doc.page_content for doc in similar_chunks])
        try:
            response = model1(context=context, query=query, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
            # print("\n[Model Output]:\n", response.strip(), "\n")
            chat = response

        except Exception as e:
            print(f"[Error during inference]: {e}")
        torch.cuda.empty_cache()
        break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

<think>

Step 1: Extracted Legal Keywords  
[List all legal terms, phrases, and document structures found directly in the context.]

- W.P.(C) No. (Court Case Number)
- IA No. (Intervention Application Number)
- Intervention/Application
- Ex-Parte Stay
- Permission to File
- Lengthy List of Dates
- Grant of Interim Relief
- Clarification/Direction
- Early Hearing Application
- Intervention/Impleadment
- Criminal law
- Trade and commerce
- Price control
- Union List
- Concurrent List
- State List
- Article 19
- Article 35(C)
- CO (Circular Order)
- Marginal note
- Section
- Interpretation
- Constitution
- Finance
- Property
- Contracts
- Suits

Step 2: Short Legal Summary  
[Summarize the context in 2–3 lines using legal wording, sticking strictly to the content.]

The document appears to be a collection of court case numbers and intervention applications related to various legal matters, including constitutional interpretation, criminal la

In [8]:
if __name__ == '__main__':
    chat = ""
    
    while True:
        query = """Retrieve the most informative sections of this legal document that can help determine its legal classification. Prioritize segments containing legal terms, section headings, clauses, named parties, obligations, dates, and governing laws. Specifically look for any signs related to:

1. Contractual – references to B2B or B2C agreements, terms and conditions, service-level agreements, non-disclosure agreements, etc.
2. Regulatory and Compliance – internal audit language, compliance obligations, policy enforcement, external submissions to government bodies.
3. Intellectual Property Rights (IPR) – mentions of patents, copyrights, trademarks, trade secrets, licenses, or intellectual asset ownership.
4. Litigation and Dispute – phrases like "court order", "petition", "judgment", "arbitration", "claim", or "legal dispute".
5. Property and Real Estate – lease terms, ownership transfers, land records, mortgage agreements, property deeds.
6. Corporate and Governance – company founding documents, board resolutions, shareholder agreements, filings, annual reports.
7. Employment and HR – employment agreements, HR policies, termination letters, NDAs, resignation or separation terms.
8. Finance and Banking – loan terms, guarantee clauses, investment contracts, collateral agreements, promissory notes.
9. Government and Public Laws – public notices, licenses, Right to Information (RTI) references, certificates issued by authorities.
10. Personal and Civil Legal Documents – ID proofs, marriage/divorce certificates, affidavits, inheritance documents, family law terms.
11. Others – anything that doesn’t fit above but still contains legal relevance.

Return the most relevant parts verbatim that clearly show which category the document may belong to.
"""

        if query.lower() == 'exit':
            print("Thanks for using the classifier!")
            break
        similar_chunks = db.similarity_search(query, k=5)
        context = "\n\n".join([doc.page_content for doc in similar_chunks])
        try:
            response = model1(context=context, query=query, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
            # print("\n[Model Output]:\n", response.strip(), "\n")
            chat = response

        except Exception as e:
            print(f"[Error during inference]: {e}")
        torch.cuda.empty_cache()
        break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

<think>

Step 1: Extracted Legal Keywords  
[List all legal terms, phrases, and document structures found directly in the context.]

- Arbitration
- Arbitration Petition
- Arbitrator
- Arbitration & Conciliation Act, 1996
- Term Sheet
- Definitive Agreements
- Binding contract
- Non-binding document
- Preamble
- Board Resolution
- Authorization
- Consent
- Conditions
- Obligations
- Closing obligations
- Closing the Transaction
- Definitive Documents
- Clause
- Annexure
- Conditions precedent
- Conditions subsequent
- Arbitration clause
- Section 11(6) of the Arbitration & Conciliation Act, 1996

Step 2: Short Legal Summary  
[Summarize the context in 2–3 lines using legal wording, sticking strictly to the content.]

The document pertains to an arbitration proceeding between Claimants and Respondent regarding a Term Sheet dated 25.11.2015, which is disputed as to whether it is a binding or non-binding document.

Step 3: Multi-Angle Rethink

In [4]:
if __name__ == '__main__':
    chat = ""
    
    while True:
        query = """Retrieve the most informative sections of this legal document that can help determine its legal classification. Prioritize segments containing legal terms, section headings, clauses, named parties, obligations, dates, and governing laws. Specifically look for any signs related to:

1. Contractual – references to B2B or B2C agreements, terms and conditions, service-level agreements, non-disclosure agreements, etc.
2. Regulatory and Compliance – internal audit language, compliance obligations, policy enforcement, external submissions to government bodies.
3. Intellectual Property Rights (IPR) – mentions of patents, copyrights, trademarks, trade secrets, licenses, or intellectual asset ownership.
4. Litigation and Dispute – phrases like "court order", "petition", "judgment", "arbitration", "claim", or "legal dispute".
5. Property and Real Estate – lease terms, ownership transfers, land records, mortgage agreements, property deeds.
6. Corporate and Governance – company founding documents, board resolutions, shareholder agreements, filings, annual reports.
7. Employment and HR – employment agreements, HR policies, termination letters, NDAs, resignation or separation terms.
8. Finance and Banking – loan terms, guarantee clauses, investment contracts, collateral agreements, promissory notes.
9. Government and Public Laws – public notices, licenses, Right to Information (RTI) references, certificates issued by authorities.
10. Personal and Civil Legal Documents – ID proofs, marriage/divorce certificates, affidavits, inheritance documents, family law terms.
11. Others – anything that doesn’t fit above but still contains legal relevance.

Return the most relevant parts verbatim that clearly show which category the document may belong to.
"""

        if query.lower() == 'exit':
            print("Thanks for using the classifier!")
            break
        similar_chunks = db.similarity_search(query, k=5)
        context = "\n\n".join([doc.page_content for doc in similar_chunks])
        try:
            response = model1(context=context, query=query, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
            # print("\n[Model Output]:\n", response.strip(), "\n")
            chat = response

        except Exception as e:
            print(f"[Error during inference]: {e}")
        torch.cuda.empty_cache()
        break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

<think>

Step 1: Extracted Legal Keywords  
[List all legal terms, phrases, and document structures found directly in the context.]

- License
- Agreement
- Representations and Warranties
- Limitations of Liability
- Disclaimer
- Indemnification
- Force Majeure
- Governing Law and Forum
- Assignment
- Severability
- Waiver
- Termination
- Notice
- Consent
- Assignment
- Sale
- Merger
- Business Combination
- Liquidation
- Trustee
- Receiver
- Jurisdiction
- Venue
- Arbitration
- Injunction
- Damages
- Breach
- Contract
- Obligation
- Enforceability
- Validity
- Intellectual Property
- Trademark
- Copyright
- Patent
- Right of Privacy
- Defamation
- Exclusivity
- Jurisdiction
- Venue
- Governing Law

Step 2: Short Legal Summary  
[Summarize the context in 2–3 lines using legal wording, sticking strictly to the content.]

This document is an agreement between Zynga Inc. and Allied Esports Entertainment, Inc. regarding the licensing of intell

In [4]:
if __name__ == '__main__':
    chat = ""
    
    while True:
        query = """Retrieve the most informative sections of this legal document that can help determine its legal classification. Prioritize segments containing legal terms, section headings, clauses, named parties, obligations, dates, and governing laws. Specifically look for any signs related to:

1. Contractual – references to B2B or B2C agreements, terms and conditions, service-level agreements, non-disclosure agreements, etc.
2. Regulatory and Compliance – internal audit language, compliance obligations, policy enforcement, external submissions to government bodies.
3. Intellectual Property Rights (IPR) – mentions of patents, copyrights, trademarks, trade secrets, licenses, or intellectual asset ownership.
4. Litigation and Dispute – phrases like "court order", "petition", "judgment", "arbitration", "claim", or "legal dispute".
5. Property and Real Estate – lease terms, ownership transfers, land records, mortgage agreements, property deeds.
6. Corporate and Governance – company founding documents, board resolutions, shareholder agreements, filings, annual reports.
7. Employment and HR – employment agreements, HR policies, termination letters, NDAs, resignation or separation terms.
8. Finance and Banking – loan terms, guarantee clauses, investment contracts, collateral agreements, promissory notes.
9. Government and Public Laws – public notices, licenses, Right to Information (RTI) references, certificates issued by authorities.
10. Personal and Civil Legal Documents – ID proofs, marriage/divorce certificates, affidavits, inheritance documents, family law terms.
11. Others – anything that doesn’t fit above but still contains legal relevance.

Return the most relevant parts verbatim that clearly show which category the document may belong to.
"""

        if query.lower() == 'exit':
            print("Thanks for using the classifier!")
            break
        similar_chunks = db.similarity_search(query, k=5)
        context = "\n\n".join([doc.page_content for doc in similar_chunks])
        try:
            response = model1(context=context, query=query, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
            # print("\n[Model Output]:\n", response.strip(), "\n")
            chat = response

        except Exception as e:
            print(f"[Error during inference]: {e}")
        torch.cuda.empty_cache()
        break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

<think>

Step 1: Extracted Legal Keywords  
[List all legal terms, phrases, and document structures found directly in the context.]

* Petitioners
* Respondent
* Writ Petition
* Coram
* Hon'ble Mr. Justice
* Judgment (Oral)
* Release Medical Board (RMB)
* Disability Pension
* Entitlement Rules
* Supreme Court
* Tribunal
* AFT (Armed Forces Tribunal)
* Onus
* Claimant
* Military Service
* Medical Category
* Low Medical Category A4 G2(P)
* Hypertension
* Primary Hypertension
* Medical Board Report
* Personal Statement
* Commanding Officer
* Part-III of the RMB Report
* Field/HAA or CIOPS / Active operation area
* AFT (Armed Forces Tribunal)
* Entitlement Rules
* Supreme Court
* RMB (Release Medical Board)
* RMB Report
* Medical Category
* Low Medical Category A4 G2(P)
* Hypertension
* Primary Hypertension
* Medical Board Report
* Personal Statement
* Commanding Officer
* Part-III of the RMB Report
* Field/HAA or CIOPS / Active operation area

# Further Classification model

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import warnings

warnings.filterwarnings('ignore')
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,StoppingCriteria, StoppingCriteriaList
import torch
import threading
import re

In [9]:
class Stream(StoppingCriteria):
    def __init__(self, tokenizer, stop_token_id=None):
        self.tokenizer = tokenizer
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        new_token_id = input_ids[0, -1].item()
        if self.stop_token_id is not None and new_token_id == self.stop_token_id:
            return True
        token_text = self.tokenizer.decode([new_token_id], skip_special_tokens=False)
        print(token_text, end="", flush=True)
        return False
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def loading_pdf(path="/home/pa2/ml/Document-Classification/files/1_D:\BPL\BLS\Document\Arbitration Award of Zostel Hospitality relied upon by the Claimant.txt"):
    """Loads a text file and splits it into chunks."""
    loader = TextLoader(path, encoding="utf-8")
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=100
    )
    docs = text_splitter.split_documents(docs)
    
    return docs
    
def faiss1(docs):
    model=HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2"
    )
    db=FAISS.from_documents(docs,model)
    return db
def model1(context,query,chat,model,tokenizer,maxlen=3000):
    
    message = [{
        "role": "user",
        "content": f"""You are a legal assistant trained to classify legal documents with zero tolerance for guessing. You must follow the rules below strictly:

RULES:
1. Do NOT make assumptions or use external knowledge.
2. ONLY use exact words, legal terms, and structures from the given context.
3. You MUST explain your classification using legal terminology from the context.
4. You MUST follow the exact output format below.
5. Return one and only one category from the list below.

LEGAL CATEGORIES with EXAMPLES:

1. Contractual – (e.g., B2B agreements, purchase orders, NDAs, service contracts)  
2. Regulatory and Compliance – (e.g., compliance reports, internal audits, regulatory filings)  
3. Intellectual Property Rights (IPR) – (e.g., patents, trademarks, copyrights, design rights, trade secrets)  
4. Litigation and Dispute – (e.g., court filings, legal notices, arbitration claims, judgments)  
5. Property and Real Estate – (e.g., lease agreements, sale deeds, land registration, rent contracts)  
6. Corporate and Governance – (e.g., company formation docs, resolutions, board meeting records, annual filings)  
7. Employment and Human Resources (HR) – (e.g., employee contracts, HR policies, separation letters)  
8. Finance and Banking – (e.g., loan agreements, collateral deeds, investment deals, bank guarantees)  
9. Government and Public Laws – (e.g., RTI responses, government licenses, public notices, certificates)  
10. Personal and Civil Legal Documents – (e.g., ID proofs, affidavits, marriage/divorce documents, family law papers)  
11. Others – (Only if no other category fits)


---
MANDATORY RULE MUST FOLLOW :-
-> **PROPERLY SHOW ALL THE STEPS IN <think> SECTION** (MANDATORY)
<context>
{context}
</context>

---

<think>Steps

Step 1: Extracted Legal Keywords →  
[List all legal terms, phrases, and document structures found directly in the context.]

Step 2: Short Legal Summary →  
[Summarize the context in 2–3 lines using legal wording, sticking strictly to the content.]

Step 3: Multi-Angle Rethinking →  
[Restate the summary from multiple angles: formal, casual, detailed, abstract, technical.]

Step 4: Category Matching →  
[Compare summary + keywords against each category. Write your thought process for every match or mismatch.]

Step 5: Eliminate Unrelated Categories →  
[List all categories that clearly do NOT apply and briefly explain why.]

Step 6: Deep Legal Reasoning →  
[Based on the remaining options, explain in detail why one specific category is the strongest fit using legal logic.]

Step 7: Final Justification →  
[Final 1-2 lines defending your classification using exact phrases or structure from the context.]

Step 8: Sub-Category Identification →
Refer to the legal sub-category hierarchy provided. Using the keywords, structure, and context of the document:

→ If confidently matched: Identify the sub-category under the selected main category (e.g., Contractual → General Commercial Contracts → Sales Agreements).
→ If only the second level (e.g., General Commercial Contracts) is confidently identifiable and not the third level (e.g., Sales Agreements), stop at the second level.
→ If sub-categorization is ambiguous or inconsistent with the context, return only the main category determined in Step 7.

Write your analysis as follows:
• Sub-Category Options Considered: [List possible sub-categories under the selected main category]
• Reasoning for Chosen Sub-Category (if any): [Use legal terms and document structure from context + .txt reference]
• Final Sub-Category Classification (max depth = 3 levels): [Write final classification path OR stop at level 2 if unsure]

</think>

<answer>
Choose ONLY ONE from this list (copy-paste one line only):

1. Contractual  
2. Regulatory and Compliance  
3. Intellectual Property Rights (IPR)  
4. Litigation and Dispute  
5. Property and Real Estate  
6. Corporate and Governance  
7. Employment and Human Resources (HR)  
8. Finance and Banking  
9. Government and Public Laws  
10. Personal and Civil Legal Documents  
11. Others
</answer>


Question: What is the correct legal category for this document?
"""


    }]

    prompt=tokenizer.apply_chat_template(message,tokenize=False)
    input_ids=tokenizer(prompt,return_tensors='pt').to("cuda")
    stopping_criteria = StoppingCriteriaList([Stream(tokenizer)])
    with torch.no_grad():
        response=model.generate(input_ids=input_ids['input_ids'],
                                attention_mask=input_ids['attention_mask'],
                                stopping_criteria=stopping_criteria,
                                max_new_tokens=maxlen,
                                use_cache=True,
                               temperature=0.1,)
    answer=tokenizer.decode(response[0])
    print()
    return answer

def model2(context,query,chat,model,tokenizer,maxlen=3000):
    
    message = [{
        "role": "user",
        "content": f"""You are a legal assistant trained to classify legal documents with zero tolerance for guessing. You must follow the rules below strictly:

RULES:
1. Do NOT make assumptions or use external knowledge.
2. ONLY use exact words, legal terms, and structures from the given context.
3. You MUST explain your classification using legal terminology from the context.
4. You MUST follow the exact output format below.
5. Return one and only one category from the list below.

LEGAL CATEGORIES with EXAMPLES:
{query}


---
MANDATORY RULE MUST FOLLOW :-
-> **PROPERLY SHOW ALL THE STEPS IN <think> SECTION** (MANDATORY)
<context>
{context}
</context>

---

<think>Steps

Step 1: Extracted Legal Keywords →  
[List all legal terms, phrases, and document structures found directly in the context.]

Step 2: Short Legal Summary →  
[Summarize the context in 2–3 lines using legal wording, sticking strictly to the content.]

Step 4: SUB Category Matching →  
[Compare summary + keywords against each category. Write your thought process for every match or mismatch.]

Step 5: Eliminate Unrelated SUB Categories →  
[List all categories that clearly do NOT apply and briefly explain why.]

Step 6: Deep Legal Reasoning →  
[Based on the remaining options, explain in detail why one specific sub category is the strongest fit using legal logic.]

Step 7: Final Justification →  
[Final 1-2 lines defending your classification using exact phrases or structure from the context.]


</think>

<answer>
(whatever is the best matched category)
</answer>


Question: What is the correct legal category for this document?
"""


    }]

    prompt=tokenizer.apply_chat_template(message,tokenize=False)
    input_ids=tokenizer(prompt,return_tensors='pt').to("cuda")
    stopping_criteria = StoppingCriteriaList([Stream(tokenizer)])
    with torch.no_grad():
        response=model.generate(input_ids=input_ids['input_ids'],
                                attention_mask=input_ids['attention_mask'],
                                stopping_criteria=stopping_criteria,
                                max_new_tokens=maxlen,
                                use_cache=True,
                               temperature=0.1,)
    answer=tokenizer.decode(response[0])
    print()
    return answer

def loading_pdf1(path):
    """Loads a PDF or text file and splits it into chunks."""
    if path.endswith(".txt"):
        loader = TextLoader(path, encoding="utf-8")
    elif path.lower().endswith(".pdf"):
        loader = PyPDFLoader(path)
    else:
        raise ValueError("Unsupported file type. Only .txt and .pdf are supported.")

    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=20
    )
    docs = text_splitter.split_documents(docs)
    return docs
# docs=loading_pdf()
docs=loading_pdf1("/home/pa2/ml/Document-Classification/files/Section_12_in_The_Contempt_Of_Courts_Act_1971.PDF")
db=faiss1(docs)

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Specify how much memory can be used by GPU and CPU
max_memory = {
    0: "7GiB",     # GPU 0 — use an integer instead of "cuda:0"
    "cpu": "30GiB"
}


model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",  # Huggingface will decide based on max_memory
    max_memory=max_memory,
    trust_remote_code=True
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
def txt_address(response):
    if "Contractual" in response:
        return 1
    elif "Regulatory" in response:
        return 2
    elif "Intellectual" in response:
        return 3
    elif "Litigation" in response:
        return 4
    elif "Property" in response:
        return 5
    elif "Corporate" in response:
        return 6
    elif "Employment" in response:
        return 7
    elif "Finance" in response:
        return 8
    elif "Government" in response:
        return 9
    elif "Personal" in response:
        return 10
    else:
        return 11

In [10]:
if __name__ == '__main__':
    address={
        1:"/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/Sub Class files/1.Contractual Document Sub Categories.txt",
        2:"/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/Sub Class files/2.Regulatory and Compliance Sub Categories.txt",
        3:"/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/Sub Class files/3.IPR Categories.txt",
        4:"/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/Sub Class files/4.Litigation and Dispute Categories.txt",
        5:"/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/Sub Class files/5.Property and Real Estate.txt",
        6:"/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/Sub Class files/6.Corporate and Governance Categories.txt",
        7:"/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/Sub Class files/7.HR Document categories.txt",
        8:"/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/Sub Class files/8.Finance and Banking Categories.txt",
        9:"/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/Sub Class files/9.Government and public laws.txt",
        10:"/home/pa2/ml/Document-Classification/CLASSIFICATION FILES/Sub Class files/10.Personal and Civil Legal Documents.txt"
    }

    query = """Retrieve the most informative sections of this legal document that can help determine its legal classification. Prioritize segments containing legal terms, section headings, clauses, named parties, obligations, dates, and governing laws. Specifically look for any signs related to:

1. Contractual – references to B2B or B2C agreements, terms and conditions, service-level agreements, non-disclosure agreements, etc.
2. Regulatory and Compliance – internal audit language, compliance obligations, policy enforcement, external submissions to government bodies.
3. Intellectual Property Rights (IPR) – mentions of patents, copyrights, trademarks, trade secrets, licenses, or intellectual asset ownership.
4. Litigation and Dispute – phrases like "court order", "petition", "judgment", "arbitration", "claim", or "legal dispute".
5. Property and Real Estate – lease terms, ownership transfers, land records, mortgage agreements, property deeds.
6. Corporate and Governance – company founding documents, board resolutions, shareholder agreements, filings, annual reports.
7. Employment and HR – employment agreements, HR policies, termination letters, NDAs, resignation or separation terms.
8. Finance and Banking – loan terms, guarantee clauses, investment contracts, collateral agreements, promissory notes.
9. Government and Public Laws – public notices, licenses, Right to Information (RTI) references, certificates issued by authorities.
10. Personal and Civil Legal Documents – ID proofs, marriage/divorce certificates, affidavits, inheritance documents, family law terms.
11. Others – anything that doesn’t fit above but still contains legal relevance.

Return the most relevant parts verbatim that clearly show which category the document may belong to.
"""
    chat=""
    if query.lower() == 'exit':
        print("Thanks for using the classifier!")
    similar_chunks = db.similarity_search(query, k=3)
    context = "\n\n".join([doc.page_content for doc in similar_chunks])
    try:
        response = model1(context=context, query=query, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)

    except Exception as e:
        print(f"[Error during inference]: {e}")
    # torch.cuda.empty_cache()
    # matches = [match.start() for match in re.finditer("<answer>", response)]
    # response1=response[matches[-1]+10:]
    # key=txt_address(response1)
    # print(key)
    # if key!=11:
    #     with open(address[key], "r", encoding="utf-8") as file:
    #         new_context = file.read()
    #     response1 = model2(context=context, query=new_context, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
        

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

<think>

Step 1: Extracted Legal Keywords  
[Legal terms, phrases, and document structures found directly in the context: contempt of court, contempt, punishment, imprisonment, fine, civil contempt, civil prison, company, director, manager, secretary, officer, contempt of court, undertaking, guilty, apology, sub-section, sub-section (1), sub-section (4), sub-section (5), Explanation.]

Step 2: Short Legal Summary  
[The document outlines the punishment for contempt of court, including imprisonment, fine, or both, and the liability of companies and their officers for contempt committed by them.]

Step 3: Multi-Angle Rethinking  
[Formal: The document prescribes the punishment for contempt of court. Casual: It's about being naughty to the court and getting in trouble. Detailed: The document outlines the specific punishments for contempt of court, including imprisonment and fines. Abstract: A legal framework for dealing with court disobedienc

In [8]:
matches = [match.start() for match in re.finditer("<answer>", response)]
response1=response[matches[-1]+10:]
key=txt_address(response1)
print(key)
if key!=11:
    with open(address[key], "r", encoding="utf-8") as file:
        new_context = file.read()
    response1 = model2(context=context, query=new_context, chat=chat, model=model, tokenizer=tokenizer,maxlen=2048)
        

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


9
<|start_header_id|>assistant<|end_header_id|>

<think>

Step 1: Extracted Legal Keywords  
- Contempt of court
- Punishment
- Imprisonment
- Fine
- Apology
- Civil contempt
- Detention in civil prison
- Company
- Director
- Manager
- Secretary
- Officer
- Firm
- Association of individuals
- Body corporate
- Subordinate court
- Undertaking given to a court

Step 2: Short Legal Summary  
The Contempt of Courts Act, 1971, prescribes punishments for contempt of court, including imprisonment, fine, or both, and provides for the liability of companies and their officers for contempt committed by them.

Step 4: Category Matching  
Comparing the summary and keywords against each category and subdomain:

- Legislative Documents: No match, as the document is not a proposed legislation, nor is it a record of legislative proceedings.
- Executive Orders and Regulations: No match, as the document is not an executive order, proclamation, regulation, or guidance document.
- Judicial Decisions on Pub

In [9]:
print(new_context)

Subdomains:
	1. Legislative Documents: (Documents created by legislative bodies such as parliaments, congresses, and state legislatures)
	    1.1 Legislative Documents Categories:
		(i) Statutes/Acts of Parliament/Laws: Formal written laws enacted by a legislative body at the national, state, or local level.
		(ii) Bills: Proposed legislation introduced in a legislative body that has not yet been enacted into law.
		(iii) Legislative Records: Official transcripts and records of the proceedings of legislative sessions (e.g., Hansard, Congressional Record).
		(iv) Amendments to Legislation: Formal changes or additions made to existing statutes or laws.
		(v) Ordinances: Laws passed by local government bodies such as cities or municipalities.

	2. Executive Orders and Regulations: (Documents issued by the executive branch of government and its agencies)
	    2.1 Executive Orders and Regulations Categories:
		(i) Executive Orders: Directives issued by the head of the executive branch (e.g.

In [19]:
"<answer>\n4. Litigation and Dispute\n</answer><|eot_id|>" in response

True

In [6]:
import re

